In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F

with open('../poems.txt', 'r', encoding='utf-8') as f:
    text = f.read()
#characters that appear in the text
chars = sorted(list(set(text)))
vocab_size = len(chars)

# hyperparameters
parameters = {'block_size':8, 'n_embd': 32, 'n_head':4, 'n_layer':4, 'dropout':0.2, 'vocab_size':vocab_size}
batch_size = 32
block_size = parameters['block_size']
max_iters = 5000
eval_interval = 500
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
# ------------

#mapping of the chars to ints and vice-versa
stoi = { ch:i for i,ch in enumerate(chars)}
itos = { i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

#splitting the data
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

#data loading
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i: i+block_size] for i in ix])
    y = torch.stack([data[i+1: i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x,y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y = get_batch(split)
            logits, loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out
    


In [3]:
#characters that appear in the text
chars = sorted(list(set(text)))
vocab_size = len(chars)
#mapping of the chars to ints and vice-versa
stoi = { ch:i for i,ch in enumerate(chars)}
itos = { i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

#splitting the data
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

#data loading
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i: i+block_size] for i in ix])
    y = torch.stack([data[i+1: i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x,y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y = get_batch(split)
            logits, loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


In [4]:
from transformer import GPTLanguageModel


In [5]:
model = GPTLanguageModel(parameters)
m = model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)


In [6]:
#training
for iter in range(max_iters):

    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    #sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 5.0453, val loss 5.0467
step 500: train loss 2.4802, val loss 2.5847
step 1000: train loss 2.3571, val loss 2.4519
step 1500: train loss 2.3056, val loss 2.4101
step 2000: train loss 2.2695, val loss 2.3666
step 2500: train loss 2.2323, val loss 2.3387
step 3000: train loss 2.2036, val loss 2.3234
step 3500: train loss 2.1953, val loss 2.3130
step 4000: train loss 2.1742, val loss 2.2895
step 4500: train loss 2.1663, val loss 2.2696


In [7]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))


Em arlita. Tfomem.
Ferno-ó eu e.. Com veudos;] apre feera sont pe geu-mestou,
Deusa efivevazão onte elando vei,h, e esmo rost mar, not é naisco for, toderas,
  lá é, e so baridão que da e feinha,
E essoa
Ó hye que cemá dires!
 o heevat o qué de no que nada folto..


Ostadio, que desfinda.
Desia dea uma anto
Nes ou coluim, exda que úlite, a ale-me, ou brantos, huis-mo potturm vicosseto pras,
To desque se que se as eu, é a juis ere dadas mentada, —

Navender necígife bouvora!
Pessos tria» a nos a 


In [ ]:
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

In [ ]:
torch.save(model, 'model.pt')